In [ ]:
import pandas as pd

In [ ]:
def evaluate_backtest(df, position_size=1):
    """
    Evaluates backtest results and calculates performance metrics.

    Parameters:
        df (pd.DataFrame): DataFrame with 'Trade_Status', 'Exit_Reason', and 'Close' columns.
        position_size (int): Number of shares per trade.

    Returns:
        result_df (pd.DataFrame): DataFrame with trade-level details.
        summary (dict): Summary of performance metrics.
    """
    trades = []
    in_trade = False
    entry_index, exit_index = None, None
    entry_price, exit_price = None, None
    direction = None

    for i in range(len(df)):
        row = df.iloc[i]

        if row['Trade_Status'] in ['Entered Long', 'Entered Short']:
            in_trade = True
            entry_index = df.index[i]
            entry_price = row['Close']
            direction = 'Long' if 'Long' in row['Trade_Status'] else 'Short'

        elif row['Trade_Status'] == 'Exited' and in_trade:
            exit_index = df.index[i]
            exit_price = row['Close']
            exit_reason = row['Exit_Reason']

            # Calculate PnL in points
            if direction == 'Long':
                points = exit_price - entry_price
            else:
                points = entry_price - exit_price

            amount = points * position_size

            trades.append({
                'Entry Time': entry_index,
                'Exit Time': exit_index,
                'Direction': direction,
                'Entry Price': round(entry_price, 2),
                'Exit Price': round(exit_price, 2),
                'Points': round(points, 2),
                'Amount': round(amount, 2),
                'Exit Reason': exit_reason
            })

            in_trade = False  # Reset for next trade

    result_df = pd.DataFrame(trades)

    # === Summary Metrics ===
    total_trades = len(result_df)
    winning_trades = result_df[result_df['Amount'] > 0]
    losing_trades = result_df[result_df['Amount'] < 0]
    total_return = result_df['Amount'].sum()
    win_rate = len(winning_trades) / total_trades * 100 if total_trades > 0 else 0
    avg_win = winning_trades['Amount'].mean() if not winning_trades.empty else 0
    avg_loss = losing_trades['Amount'].mean() if not losing_trades.empty else 0
    profit_factor = -winning_trades['Amount'].sum() / losing_trades['Amount'].sum() if not losing_trades.empty else float('inf')

    summary = {
        'Total Trades': total_trades,
        'Winning Trades': len(winning_trades),
        'Losing Trades': len(losing_trades),
        'Win Rate (%)': round(win_rate, 2),
        'Total Return': round(total_return, 2),
        'Average Win': round(avg_win, 2),
        'Average Loss': round(avg_loss, 2),
        'Profit Factor': round(profit_factor, 2)
    }

    return result_df, summary